Exploring Airbnb Data For Vancouver

Data Source: http://insideairbnb.com/get-the-data

Questions:

- #1: Does higher price mean higher rating? What if we bin listings according to price: Highest, High, Medium, Low, Lowest. And then review the correlations to ratings in each group?
- #2: What if bin listings according to price category and date? What are the price trends for each category throughout the seasons?
- #3: What if bin listing according to neighborhood. Are there any trends in each neighborhood? Proportion of superhosts in each neighborhood?
- #4:  Aside from review what characteristics can be helpful in predicting a hosts rating?
- #5: Can those same factors be used to classify whether a host is a superhost?

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

In [ ]:
pd.set_option('display.max_columns',200)
pd.set_option('display.max_rows',500)

Load Data

In [ ]:
calendar_dfs = []
listings_dfs = []
reviews_dfs = []

for subdir, dirs, files in  os.walk("./vancouver_data/"):
    for file in files:
        pth = os.path.join(subdir,file)
        if "calendar" in file:
            c_data = pd.read_csv(pth,compression='gzip')
            calendar_dfs.append(c_data)
        elif "listings" in file:
            l_data = pd.read_csv(pth)
            listings_dfs.append(l_data)
        elif "reviews" in file:
            r_data = pd.read_csv(pth)
            reviews_dfs.append(r_data)

calendar_df = pd.concat(calendar_dfs,ignore_index=True)
listings_df = pd.concat(listings_dfs,ignore_index=True)
reviews_df = pd.concat(reviews_dfs,ignore_index=True)

Preview Data

In [ ]:
def explore_data(df):
    print(">>> New DataFrame <<< \n")
    print("DataFrame preview: \n",df.head())
    print("*"*20)
    print("Shape: \n", df.shape)
    print("*"*20)
    print("Statistics: \n",df.describe())
    print("*"*20)
    print("Data types: \n",df.dtypes)
    print("*"*20)
    print("Columns: \n",df.columns)
    print("*"*20)
    print("Index: \n",df.index)
    print("*"*20)
    print("Missing Values: \n",df.isna().sum())

In [ ]:
# df_list = [calendar_df,listings_df,reviews_df]
# for frame in df_list:
#     explore_data(frame)

Prepare Data For Analysis

In [ ]:
cal_df = calendar_df.copy()
lst_df = listings_df.copy()
rev_df = reviews_df.copy()


Calendar DataFrame

- Duplicate listing_id as a listing may be available on various dates.
- The price of a listing_id may differ depending on date.

In [ ]:
cal_df.dtypes

In [ ]:
cal_df.isna().sum()

In [ ]:
cal_df.dropna(inplace=True)

In [ ]:
cal_df.drop(labels=['adjusted_price'],axis=1,inplace=True)

In [ ]:
cal_df['date'] = pd.to_datetime(calendar_df['date'])

In [ ]:
cal_df['price'] = cal_df['price'].str.slice(start=1)
cal_df['price'] = cal_df['price'].replace(',','',regex=True)
cal_df['price'] = pd.to_numeric(cal_df['price'], errors='coerce')

In [ ]:
cal_df.shape

In [ ]:
cal_df['available'].value_counts()

In [ ]:
cal_df = cal_df[cal_df['available'] == 't']

In [ ]:
cal_df.shape

In [ ]:
cal_df.head()

#1: Does higher price mean higher rating? What if we bin listings according to price: Highest, High, Medium, Low, Lowest. And then review the correlations to ratings in each group?

In [ ]:
calendar = cal_df.drop(['available','date',\
                        'minimum_nights','maximum_nights'],axis=1)
calendar.head()

In [ ]:
calendar['price_category'] = pd.qcut(calendar['price'],q=5,\
                        labels=['Lowest','Low','Medium','High','Highest']) 

In [ ]:
calendar.head()

In [ ]:
ratings = lst_df[['id','review_scores_rating']].dropna(axis=0)

In [ ]:
calendar_join_ratings = pd.merge(left=calendar,\
                                right=ratings,\
                                left_on="listing_id",\
                                right_on="id")

In [ ]:
calendar_join_ratings.head()

In [ ]:
joined = calendar_join_ratings.drop(['listing_id','id'],axis=1)
joined.head()

In [ ]:
price_category_grouped = joined.groupby(['price_category'],as_index=False)

In [ ]:
for name,grp in price_category_grouped:
    print(name)
    print(grp[['price','review_scores_rating']].corr())
    print("*"*20)

No significant correlations in the different price groups. As as a sanity check let's review the correlation between price and rating for all listings.

In [ ]:
calendar_join_ratings[['price','review_scores_rating']].corr()

So the price, or from another point of view the cost, has little to do with the rating.

#2: What if we group listings according not only according to price but also date? 

Do prices fluctuate depending on the season for the listings in the various price categories (Lowest, Low, Medium, High, Highest)?

In [ ]:
cal_df.head()

In [ ]:
date_df = cal_df.drop(['available','minimum_nights','maximum_nights'],axis=1)

In [ ]:
date_df['month'] = date_df['date'].dt.month

In [ ]:
date_df['price_category'] = pd.qcut(date_df['price'],q=5,\
                        labels=['Lowest','Low','Medium','High','Highest']) 

In [ ]:
date_grouped = date_df.groupby(['price_category','month'],as_index=False)

In [ ]:
# for name,grp in date_grouped:
#     print(name)
#     print(date_grouped.get_group(name))

# Output:
# ('Lowest', 1)
# ('Lowest', 2)
# ('Lowest', 3)
# ...

In [ ]:
categories_data = {'Lowest':[],'Low':[],'Medium':[],'High':[],'Highest':[] }

In [ ]:
for name,grp in date_grouped:
    # month_num = name[1]
    # num_listings = grp['listing_id'].count()
    median_price = grp['price'].median()
    price_cat = name[0]
    categories_data[price_cat].append(median_price)

In [ ]:
summary_df = pd.DataFrame(data=categories_data,\
                          columns=categories_data.keys(),\
                            index=[x for x in range(1,13)])

summary_df

In [ ]:
summary_df.max()

In [ ]:
# def highlight_max(input_df):
#     print(type(input_df))
#     styled_df = input_df.copy()
#     styled_df[styled_df ==] = 'background-color: red'
#     return styled_df

def highlight_max(input_df):
    color = "red"
    attr = "background-color: {}".format(color)
    # print(type(input_df))
    is_max = input_df == input_df.max()
    return pd.DataFrame(np.where(is_max,attr,''),\
            index=input_df.index,\
            columns=input_df.columns )

summary_df.style.apply(highlight_max,axis=None) #subset=['Highest']

So as you can see the per night price does not fluctuate much throughout the year. This could be a sign that the demand is consistent throughout the year. And there is enough supply to meet incoming tourists, so no price surge.

In [ ]:
for col in summary_df.columns:
    price_range = summary_df[col].max() - summary_df[col].min()
    mean_price = summary_df[col].mean()
    most_exp_months = []
    for m in summary_df[col]:
        print(m)
    print("Price Range for {col} category is ${price_range}"\
          .format(col=col,price_range=price_range))
    

The dataframe plots the median price for each price category and month.
What is the most expensive month for each category? Least expensive?

In [ ]:
sns.lineplot(data=summary_df).set_title("Median Price For Category")
plt.show()